<h4>Importing Libraries

In [191]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

In [192]:
class Tokenizer():
    def fit_on_texts(self,list_data):
        word_list = " ".join(list_data).split()
        self.word_counts = list(set(word_list))
        self.word_dict = {w: i for i, w in enumerate(self.word_counts)}
        self.number_dict = {i: w for i, w in enumerate(self.word_counts)}
        
    def texts_to_sequences(self,data):
        encoded_sequence = list()
        for item in data:
            encoded_sequence.append([self.word_dict[word] for word in item.split()])
        return encoded_sequence
    
def pad_sequences(data,padding='pre',padding_value=0):
    sequence = None
    if isinstance(data,list):
        maxlen = max(len(item) for item in data)
        
    if padding == 'pre':
        for idx in range(len(data)):
            data[idx] = [padding_value]*(maxlen-len(data[idx])) + data[idx]
    else:
        for idx in range(len(data)):
            data[idx] = data[idx]+ [padding_value]*(maxlen-len(data[idx]))
                                                    
    return data
def to_categorical(data, nb_classes):
    targets = np.array(data).reshape(-1)
    return np.eye(nb_classes)[targets]

In [195]:
class Preprocessing():
    
    def __init__(self,input_file):
        self.input_data_file = input_file
        self.data = None
        self.vocab_size = None
        self.encoded_data = None
        self.max_length = None
        self.sequences = None
        self.x = None
        self.y = None
        self.tokenizer = None
    
    def load_data(self):
        fp = open(self.input_data_file,'r')
        self.data = fp.read().splitlines()        
        fp.close()
        
    def encode_data(self):
        self.tokenizer = Tokenizer()
        self.tokenizer.fit_on_texts(self.data)
        self.encoded_data = self.tokenizer.texts_to_sequences(self.data)
        print(self.encoded_data)
        self.vocab_size = len(self.tokenizer.word_counts)+1
        
    def generate_sequence(self):
        seq_list = list()
        for item in self.encoded_data:
            l = len(item)
            for id in range(1,l):
                seq_list.append(item[:id+1])
        #print(seq_list[0])
        print(seq_list)
        self.sequences = pad_sequences(seq_list,padding='pre', padding_value=0)
        print(self.sequences)
        self.sequences = array(self.sequences)
            
    def get_data(self):
        self.x = self.sequences[:,:-1]
        self.y = self.sequences[:,-1]
        print(self.y)
        self.y = to_categorical(self.y,nb_classes=self.vocab_size)
        print("Y:{}".format(self.y))
        print("X:{}".format(self.x))
        return self.x,self.y

In [196]:
pr = Preprocessing('data.txt')
pr.load_data()
pr.encode_data()
pr.generate_sequence()
x,y = pr.get_data()

[[2, 12, 16, 4, 3, 14, 9], [17, 0, 18, 6, 19, 21], [2, 10, 8, 12, 11, 7, 15], [1, 16, 5, 20, 13]]
[[2, 12], [2, 12, 16], [2, 12, 16, 4], [2, 12, 16, 4, 3], [2, 12, 16, 4, 3, 14], [2, 12, 16, 4, 3, 14, 9], [17, 0], [17, 0, 18], [17, 0, 18, 6], [17, 0, 18, 6, 19], [17, 0, 18, 6, 19, 21], [2, 10], [2, 10, 8], [2, 10, 8, 12], [2, 10, 8, 12, 11], [2, 10, 8, 12, 11, 7], [2, 10, 8, 12, 11, 7, 15], [1, 16], [1, 16, 5], [1, 16, 5, 20], [1, 16, 5, 20, 13]]
[[0, 0, 0, 0, 0, 2, 12], [0, 0, 0, 0, 2, 12, 16], [0, 0, 0, 2, 12, 16, 4], [0, 0, 2, 12, 16, 4, 3], [0, 2, 12, 16, 4, 3, 14], [2, 12, 16, 4, 3, 14, 9], [0, 0, 0, 0, 0, 17, 0], [0, 0, 0, 0, 17, 0, 18], [0, 0, 0, 17, 0, 18, 6], [0, 0, 17, 0, 18, 6, 19], [0, 17, 0, 18, 6, 19, 21], [0, 0, 0, 0, 0, 2, 10], [0, 0, 0, 0, 2, 10, 8], [0, 0, 0, 2, 10, 8, 12], [0, 0, 2, 10, 8, 12, 11], [0, 2, 10, 8, 12, 11, 7], [2, 10, 8, 12, 11, 7, 15], [0, 0, 0, 0, 0, 1, 16], [0, 0, 0, 0, 1, 16, 5], [0, 0, 0, 1, 16, 5, 20], [0, 0, 1, 16, 5, 20, 13]]
[12 16  4  3 14  9 

In [184]:
n_step = 1 # n-1 in paper
n_hidden = 1 # h in paper
m = 1 # m in paper
n_class = pr.vocab_size
dtype = torch.FloatTensor


In [178]:
class NNLM(nn.Module):
    def __init__(self):
        super(NNLM, self).__init__()
        self.C = nn.Embedding(n_class, m)
        self.H = nn.Parameter(torch.randn(n_step * m, n_hidden).type(dtype))
        self.W = nn.Parameter(torch.randn(n_step * m, n_class).type(dtype))
        self.d = nn.Parameter(torch.randn(n_hidden).type(dtype))
        self.U = nn.Parameter(torch.randn(n_hidden, n_class).type(dtype))
        self.b = nn.Parameter(torch.randn(n_class).type(dtype))

    def forward(self, X):
        X = self.C(X)
        X = X.view(-1, n_step * m) # [batch_size, n_step * n_class]
        tanh = torch.tanh(self.d + torch.mm(X, self.H)) # [batch_size, n_hidden]
        output = self.b + torch.mm(X, self.W) + torch.mm(tanh, self.U) # [batch_size, n_class]
        return output
    
def train(x,y):
    model = NNLM()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    # Training
    for epoch in range(100):

        optimizer.zero_grad()
        output = model(x)

        # output : [batch_size, n_class], target_batch : [batch_size] (LongTensor, not one-hot)
        loss = criterion(output, y)
        if (epoch + 1)%1000 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

        loss.backward()
        optimizer.step()

In [179]:
x = Variable(torch.LongTensor(x))
y = Variable(torch.LongTensor(y))
train(x,y)

ValueError: Expected input batch_size (126) to match target batch_size (21).